In [1]:
import json
import numpy as np
from PIL import Image
from matplotlib import image
from matplotlib import pyplot
from pathlib import Path
import torchaudio
import IPython.display as ipd
from tqdm import tqdm

In [3]:
dataset = np.load(Path("data/splited_me_dataset.npz"), allow_pickle=True)
train = dataset['train'].item()
dev = dataset['dev'].item()
test = dataset['test'].item()
unseen_test = dataset['unseen_test'].item()
# images = np.load(Path("data/images_for_test_episodes.npz"), allow_pickle=True)['images'].item()

In [4]:
audio_numbers = {}
image_numbers = {}

In [5]:
with open(Path('analysis/train_numbers.txt'), 'w') as f:
    print('train')
    total_images = 0
    total_audio = 0
    for w in train:
        i = len(train[w]['images'])
        e = len(train[w]['english'])
        f.write(f'{w}\t{i}\t{e}\n')
        total_images += i
        total_audio += e
        if w not in audio_numbers: audio_numbers[w] = {'type': 'familiar', 'color': 'royalblue', 'count': 0}
        audio_numbers[w]['count'] += e
        if w not in image_numbers: image_numbers[w] = {'type': 'familiar', 'color': 'royalblue', 'count': 0}
        image_numbers[w]['count'] += i
    print(total_images, total_audio)
        
with open(Path('analysis/val_numbers.txt'), 'w') as f:
    print('val')
    total_images = 0
    total_audio = 0
    for w in dev:
        i = len(dev[w]['images'])
        e = len(dev[w]['english'])
        f.write(f'{w}\t{i}\t{e}\n')
        total_images += i
        total_audio += e
        audio_numbers[w]['count'] += e
        image_numbers[w]['count'] += i
    print(total_images, total_audio)
        
with open(Path('analysis/test_numbers.txt'), 'w') as f:
    print('test')
    total_images = 0
    total_audio = 0
    for w in test:
        i = len(test[w]['images'])
        e = len(test[w]['english'])
        f.write(f'{w}\t{i}\t{e}\n')
        print(w, i, e)
        total_images += i
        total_audio += e
        audio_numbers[w]['count'] += e
        image_numbers[w]['count'] += i

    for w in unseen_test:
        i = len(unseen_test[w]['images'])
        e = len(unseen_test[w]['english'])
        f.write(f'{w}\t{i}\t{e}\n')
        print(w, i, e)
        total_images += i
        total_audio += e
        if w not in audio_numbers: audio_numbers[w] = {'type': 'novel', 'color': 'forestgreen', 'count': 0}
        audio_numbers[w]['count'] += e
        if w not in image_numbers: image_numbers[w] = {'type': 'novel', 'color': 'forestgreen', 'count': 0}
        image_numbers[w]['count'] += i
    print(total_images, total_audio)

FileNotFoundError: [Errno 2] No such file or directory: 'analysis/train_numbers.txt'

In [ ]:
audio_labels = []
audio_values = []
audio_tag = []
audio_colors = []
image_labels = []
image_values = []
image_tag = []
image_colors = []
scale = 1.0
for w in audio_numbers:
    print(w, audio_numbers[w]['type'], audio_numbers[w]['count'], image_numbers[w]['count'])

    audio_labels.append(w)
    audio_values.append(scale*audio_numbers[w]['count'])
    audio_tag.append(audio_numbers[w]['type'])
    audio_colors.append(audio_numbers[w]['color'])
    image_labels.append(w)
    image_values.append(scale*image_numbers[w]['count'])
    image_tag.append(image_numbers[w]['type'])
    image_colors.append(image_numbers[w]['color'])

    
audio_values = np.asarray(audio_values)
image_values = np.asarray(image_values)

In [ ]:
import matplotlib.pyplot as plt

def legend_without_duplicate_labels(ax):
    handles, labels = ax.get_legend_handles_labels()
    unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    ax.legend(*zip(*unique), fontsize=25, bbox_to_anchor=(0.1,-0.05), ncol=2, frameon=False)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 20))

# y1 = np.arange(len(audio_labels))
y1 = []
for y in range(len(audio_labels)):
    if audio_labels[y] == 'dog': scale = 0.5
    else: scale = 1.0
    ax1.barh(y, scale*audio_values[y], label=audio_tag[y], color=audio_colors[y])
    ax1.text(scale*audio_values[y], y, str(audio_values[y]), fontsize=18)
    y1.append(y)
ax1.set_yticks(y1, audio_labels, fontsize=20)
ax1.set_xticks([])
ax1.set_title('Number of audio samples per class', fontsize=25)
ax1.spines[['right', 'top', 'bottom']].set_visible(False)
# for patch,color in zip(ax1.patches,audio_colors):
#     patch.set_facecolor(color)

    
y2 = []
for y in range(len(image_labels)):
    if audio_labels[y] == 'dog': scale = 0.5
    else: scale = 1.0
    ax2.barh(y, scale*image_values[y], label=image_tag[y], color=image_colors[y])
    ax2.text(scale*image_values[y], y, str(image_values[y]), fontsize=18)
    y2.append(y)
ax2.set_yticks(y2, image_labels, fontsize=20)
ax2.set_xticks([])
ax2.set_title('Number of image samples per class', fontsize=25)
ax2.spines[['right', 'top', 'bottom']].set_visible(False)
legend_without_duplicate_labels(ax2)

plt.savefig('number_of_samples_per_class.pdf', bbox_inches='tight')
plt.show()

In [ ]:
print(sorted(train))

In [ ]:
print(sorted(unseen_test))

In [ ]:
seen_image_to_dataset = {}
unseen_image_to_dataset = {}

category_ids = np.load(Path("data/coco_dataset.npz"), allow_pickle=True)['category_ids'].item()
    
annot = {}
segments_dir = Path('../../Datasets/spokencoco/panoptic_annotations_trainval2017/annotations')
for fn in segments_dir.rglob('*.json'):

    with open(fn, 'r') as f:
        l = json.load(f)
    
    for entry in l['annotations']:
        annot[entry['image_id']] =  []
        
        for e in entry['segments_info']:
            id = e['category_id']
            if id in category_ids: annot[entry['image_id']].append(category_ids[id])

for fn in segments_dir.rglob(f'*.png'):
    a = Path(fn).parent.stem
    b = Path(fn).parent.parent.stem
    if a != b: continue
    num = int(fn.stem)
    
    for c in annot[num]: 
        name = f'{c}_masked_{num}'
        new_fn = Path(f'data/images/{name}.jpg')
        
        if c in unseen_test:
            if name not in unseen_image_to_dataset: 
                unseen_image_to_dataset[name] = {
                    'dataset':'mscoco',
                    'fn': new_fn
                }
                
                
        if c in test:

            if name not in seen_image_to_dataset: 
                seen_image_to_dataset[name] = {
                    'dataset':'mscoco',
                    'fn': new_fn
                }
            


caltech = np.load(Path('data/caltech_101_dataset.npz'), allow_pickle=True)['names'].item()

for im in Path('/media/leannenortje/HDD/Datasets/caltech-101/caltech-101/101_ObjectCategories').rglob('*/*.jpg'):
    
    raw_c = im.parent.stem
    
    if raw_c in caltech: 
        c = caltech[raw_c]
        num = int(im.stem.split('_')[-1])
        name = f'{c}_masked_{num}'
        new_fn = Path(f'data/images/{c}_masked_{num}.jpg')
        if c == 'bird' and num == 335081: print(raw_c, c, name)

        if c in unseen_test: 
            if name not in unseen_image_to_dataset: 
                unseen_image_to_dataset[name] = {
                    'dataset':'caltech',
                    'fn': new_fn
                }

        elif c in test:
            if name not in seen_image_to_dataset: 
                seen_image_to_dataset[name] = {
                    'dataset':'caltech',
                    'fn': new_fn
                }

In [ ]:
for c in test:
    print(c, len(test[c]['images']))
    
    for im in test[c]['images']:
        name = Path(im).stem
        if name not in seen_image_to_dataset: print(name)

In [ ]:
for c in images:
    print(c, len(images[c]))
    
    for im in images[c]:
        name = Path(im).stem
        if name not in unseen_image_to_dataset: print(name)

In [ ]:
seen_counting = {}
seen_images = set()

for c in test:
    
    for im in test[c]['images']:
        seen_images.add(im)
        name = Path(im).stem
        
        c = name.split('_')[0]
        dataset = seen_image_to_dataset[name]['dataset']
        if c not in seen_counting: seen_counting[c] = {'mscoco': 0, 'caltech': 0}
        seen_counting[c][dataset] += 1
    
check = 0
for c in seen_counting:
    caltech = seen_counting[c]['caltech']
    mscoco = seen_counting[c]['mscoco']
    check += mscoco
    check += caltech
    print(f'{c} Caltech: {caltech} MSCOCO: {mscoco}')
print(check)

In [ ]:
unseen_counting = {}
unseen_images = set()

for c in images:

    for im in images[c]:
        if im in unseen_images: print(im)
        unseen_images.add(im)
        name = Path(im).stem
            

        c = name.split('_')[0]
        dataset = unseen_image_to_dataset[name]['dataset']
        if c not in unseen_counting: unseen_counting[c] = {'mscoco': 0, 'caltech': 0}
        unseen_counting[c][dataset] += 1

check = 0
for c in unseen_counting:
    caltech = unseen_counting[c]['caltech']
    mscoco = unseen_counting[c]['mscoco']
    check += mscoco
    check += caltech
    print(f'{c} Caltech: {caltech} MSCOCO: {mscoco}')
print(check)

In [ ]:
all_images = set()

for c in test:
    for im in test[c]['images']:
        all_images.add(str(im))
        
for c in images:
    for im in images[c]:
        all_images.add(str(im))

In [ ]:
len(all_images)

In [ ]:
len(seen_images)

In [ ]:
len(unseen_images)

In [ ]:
len(seen_images) + len(unseen_images)